# Query time weights

Weighting of particular attributes can be set and tweaked query time - no need to re-embed the whole dataset if you wish to alter some weights.

In [1]:
%pip install superlinked==17.9.2

In [2]:
import pandas as pd
from superlinked import framework as sl

pd.set_option("display.max_colwidth", 100)

In [3]:
class Paragraph(sl.Schema):
    id: sl.IdField
    body: sl.String
    like_count: sl.Integer


paragraph = Paragraph()

body_space = sl.TextSimilaritySpace(text=paragraph.body, model="sentence-transformers/all-mpnet-base-v2")
like_space = sl.NumberSpace(number=paragraph.like_count, min_value=0, max_value=100, mode=sl.Mode.MAXIMUM)
# indices can be built on top of multiple spaces as simple as that
paragraph_index = sl.Index([body_space, like_space])

In [4]:
source: sl.InMemorySource = sl.InMemorySource(paragraph)
executor = sl.InMemoryExecutor(sources=[source], indices=[paragraph_index])
app = executor.run()

In [5]:
source.put(
    [
        {
            "id": "paragraph-1",
            "body": "Glorious animals live in the wilderness.",
            "like_count": 75,
        },
        {
            "id": "paragraph-2",
            "body": "Growing computation power enables advancements in AI.",
            "like_count": 10,
        },
    ]
)

## Defining queries

Query weights can be set at definition time, therefore no need to re-embed the dataset if aim to query with altering weights.

The `body_query` here gives double weight to the text provided in the `.similar` query compared to the like count, while the `like_query` searches the other way around: likes are twice as important, hence the differing results.

In [6]:
body_query = (
    sl.Query(
        paragraph_index,
        weights={
            body_space: 1.0,
            like_space: 0.5,
        },
    )
    .find(paragraph)
    .similar(body_space, "What makes the AI industry go forward?")
)

like_query = (
    sl.Query(
        paragraph_index,
        weights={
            body_space: 0.5,
            like_space: 1.0,
        },
    )
    .find(paragraph)
    .similar(body_space, "What makes the AI industry go forward?")
)

In [7]:
body_result = app.query(body_query)

body_result.to_pandas()

,body,like_count,id,similarity_score
0,Growing computation power enables advancements in AI.,10,paragraph-2,0.312125
1,Glorious animals live in the wilderness.,75,paragraph-1,0.243343


In [8]:
like_result = app.query(like_query)

like_result.to_pandas()

,body,like_count,id,similarity_score
0,Glorious animals live in the wilderness.,75,paragraph-1,0.425985
1,Growing computation power enables advancements in AI.,10,paragraph-2,0.182733
